In [75]:
# Importações das bibliotecas do PySpark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import substring, col, sum, count, lpad, round
from typing import Tuple
from pyspark.sql.types import DoubleType

In [76]:
# Inicia ou obtém uma sessão Spark existente
spark = SparkSession.builder \
    .appName("PipelineSimplesETL") \
    .getOrCreate()

print("Sessão Spark iniciada com sucesso.")

Sessão Spark iniciada com sucesso.


In [77]:
# Define o caminho de entrada dos arquivos
caminho_clientes = '/content/dados/clientes.txt'
caminho_vendas = '/content/dados/vendas.csv'

# Define o caminho de saida dos arquivos
caminho_saida_resumo = '/content/dados/Saida/resumo_clientes'
caminho_saida_balanco = '/content/dados/Saida/balanco_produtos'

# 1. Cria funções de execução do pipeline




In [78]:
#------------------------------------------#
# Lê o arquivo de CLIENTES no formato CSV.
#------------------------------------------#
def carrega_dados_clientes(caminho: str) -> DataFrame:
    """Lê o arquivo de clientes no formato CSV."""
    print(f"Lendo dados de clientes de: {caminho}")
    return spark.read.csv(caminho, header=True, sep=',')

#-------------------------------------------#
# Lê o arquivo de VENDAS como texto bruto.
#-------------------------------------------#
def carrega_dados_vendas_raw(caminho: str) -> DataFrame:
    """Lê o arquivo de vendas como texto bruto."""
    print(f"Lendo dados brutos de vendas de: {caminho}")
    return spark.read.csv(caminho)

#--------------------------------------------------------#
# Extrai os campos do arquivo de VENDAS de largura fixa.
#--------------------------------------------------------#
def transformar_vendas(df_vendas_raw: DataFrame) -> DataFrame:
    """Extrai os campos do arquivo de vendas de largura fixa."""
    print("Transformando dados brutos de vendas em colunas.")
    vendas_estruturadas = df_vendas_raw.select(
        substring(col("_c0"), 6, 5).alias("venda_id"),
        substring(col("_c0"), 1, 5).alias("cliente_id"),
        substring(col("_c0"), 11, 5).alias("produto_id"),
        substring(col("_c0"), 16, 8).alias("valor"),
        substring(col("_c0"), 24, 8).alias("data_venda")
    )
    return vendas_estruturadas


#-------------------------------------------#
# Prepara os DataFrames para a junção (join)
#-------------------------------------------#
def preparar_dados_para_join(df_clientes: DataFrame, df_vendas: DataFrame) -> Tuple[DataFrame, DataFrame]:
    """Prepara os DataFrames para a junção (join)."""
    print("Padronizando chaves e convertendo tipos de dados.")
    clientes_ajustado = df_clientes.withColumn("cliente_id", lpad(col("cliente_id"), 5, "0"))
    vendas_ajustado = df_vendas.withColumn("valor_numeric", (col("valor").cast(DoubleType()) / 100))
    return clientes_ajustado, vendas_ajustado

#--------------------------------------------#
# Calcula as métricas de vendas por cliente.
#--------------------------------------------#
def calcula_resumo_por_cliente(df_juncao: DataFrame) -> DataFrame:
    """Calcula as métricas de vendas por cliente."""
    print("Calculando o resumo de vendas por cliente.")
    resumo = df_juncao.groupBy("cliente_id", "nome").agg(
        sum("valor_numeric").alias("total_vendas"),
        count("venda_id").alias("quantidade_vendas"),
        round(sum("valor_numeric") / count("venda_id"), 2).alias("ticket_medio")
    )
    return resumo

#--------------------------------------------#
# Calcula as métricas de vendas por produto.
#--------------------------------------------#
def calcula_balanco_por_produto(df_juncao: DataFrame) -> DataFrame:
    """Calcula as métricas de vendas por produto."""
    print("Calculando o balanço de vendas por produto.")
    balanco = df_juncao.groupBy("produto_id").agg(
        sum("valor_numeric").alias("total_vendas_produto"),
        count("venda_id").alias("quantidade_vendas_produto"),
        round(sum("valor_numeric") / count("venda_id"), 2).alias("ticket_medio_produto")
    )
    return balanco
#-----------------------------------------------------------#
# Salva um DataFrame em um único arquivo CSV com cabeçalho.
#-----------------------------------------------------------#
def carregar_dados_csv(df: DataFrame, caminho_saida: str) -> None:
    """Salva um DataFrame em um único arquivo CSV com cabeçalho."""
    print(f"Salvando dados em: {caminho_saida}")
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(caminho_saida)
    print("Dados salvos com sucesso!")

# 2. Executa as funções do pipeline

In [79]:
try:
    # 1. Leitura de dados bruto
    df_clientes_raw = carrega_dados_clientes(caminho_clientes)
    df_vendas_raw = carrega_dados_vendas_raw(caminho_vendas)

    # 2. Transformação dos dados brutos
    df_vendas_estruturado = transformar_vendas(df_vendas_raw)
    df_clientes_pronto, df_vendas_pronto = preparar_dados_para_join(df_clientes_raw, df_vendas_estruturado)

    # Junção dos dados de cliente com a base de vendas
    print("Juntando dados de clientes e vendas.")
    df_juncao_completa = df_vendas_pronto.join(df_clientes_pronto, "cliente_id", "inner")

    # Gera as saidas com as agregações
    resumo_clientes = calcula_resumo_por_cliente(df_juncao_completa)
    balanco_produtos = calcula_balanco_por_produto(df_juncao_completa)

    # Exibe os resultados na tela
    print("\n--- Resultado: Resumo por Cliente ---")
    resumo_clientes.show()

    print("\n--- Resultado: Balanço por Produto ---")
    balanco_produtos.show()

    # Extrai os resumos por Cliente e Produto.
    carregar_dados_csv(resumo_clientes, caminho_saida_resumo)
    carregar_dados_csv(balanco_produtos, caminho_saida_balanco)

except Exception as e:
    print(f"\nOcorreu um erro durante a execução do pipeline: {e}")

Lendo dados de clientes de: /content/dados/clientes.txt
Lendo dados brutos de vendas de: /content/dados/vendas.csv
Transformando dados brutos de vendas em colunas.
Padronizando chaves e convertendo tipos de dados.
Juntando dados de clientes e vendas.
Calculando o resumo de vendas por cliente.
Calculando o balanço de vendas por produto.

--- Resultado: Resumo por Cliente ---
+----------+-----------+------------+-----------------+------------+
|cliente_id|       nome|total_vendas|quantidade_vendas|ticket_medio|
+----------+-----------+------------+-----------------+------------+
|     00001| João Silva|    613810.1|                5|   122762.02|
|     00002|Maria Souza|   360006.06|                3|   120002.02|
+----------+-----------+------------+-----------------+------------+


--- Resultado: Balanço por Produto ---
+----------+--------------------+-------------------------+--------------------+
|produto_id|total_vendas_produto|quantidade_vendas_produto|ticket_medio_produto|
+-----